In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import re

In [2]:
community_Wide_2017 = pd.read_csv('../DBData/2017_-_Cities_Community_Wide_Emissions.csv')
emissions_reduction_2016 = pd.read_csv('../DBData/2016_-_Cities_Emissions_Reduction_Targets_20240207.csv')
emissions_reduction_2017 = pd.read_csv("../DBData/2017_-_Cities_Emissions_Reduction_Targets_20240207.csv")
citywide_ghg_2016 = pd.read_csv('../DBData/2016_-_Citywide_GHG_Emissions_20240207.csv')
climate_risk = pd.read_csv('../DBData/2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv')



,Account number,Organization,City,Country,Region,C40,Access,Reporting year,Accounting year,Boundary,...,Population year,GDP,GDP Currency,GDP Year,GDP Source,Average annual temperature (in Celsius)​,​Average altitude (m),​Land area (in square km),City Location,Country Location
0,49363,Nelson Mandela Bay Municipality,Nelson Mandela Bay,South Africa,Africa,NaN,Public,2017,2013-07-01 - 2014-06-30,A metropolitan area,...,2011,5.214700e+04,ZAR South African Rand,2010.0,Built Environment Performance Plan(BEPP),17.5,69.0,1950.0,"(-33.745241, 25.568108)","(-30.559482, 22.937506)"
1,31171,Ayuntamiento de Madrid,Madrid,Spain,Europe,C40,Public,2017,2014-01-01 - 2014-12-31,Administrative boundary of a local government,...,2016,1.293780e+11,EUR Euro,2015.0,Contabilidad Municipal Trimestral. Ciudad de M...,15.0,655.0,604.0,"(40.1076253, -3.3875673)","(40.463667, -3.74922)"
2,3417,New York City,New York City,USA,North America,C40,Public,2017,2015-01-01 - 2015-12-31,Administrative boundary of a local government,...,2016,1.558520e+12,USD US Dollar,2015.0,http://www.brookings.edu/research/reports2/201...,13.0,10.0,784.0,"(40.7127837, -74.0059413)","(37.09024, -95.712891)"
3,59537,"City of Denton, TX","Denton, TX",USA,North America,NaN,Public,2017,2015-01-01 - 2015-12-31,Administrative boundary of a local government,...,2015,NaN,NaN,NaN,NaN,18.2,192.0,227.8,"(33.214841, -97.133068)","(37.09024, -95.712891)"
4,35894,Ville de Montreal,Montreal,Canada,North America,C40,Public,2017,2013-01-01 - 2013-12-31,A metropolitan area,...,2013,1.154128e+08,CAD Canadian Dollar,2013.0,Institut de la statistique du Québec,6.8,17.0,500.0,"(45.5086699, -73.5539925)","(56.130366, -106.346771)"


In [3]:
temperature1 = citywide_ghg_2016[["Average annual temperature (in Celsius)​", "Measurement Year", "City Name"]]
temperature1["Measurement Year"] = pd.to_datetime(temperature1["Measurement Year"]).dt.year

temperature2 = community_Wide_2017[["Average annual temperature (in Celsius)​","Reporting year", "City"]]

temperature2 = temperature2.rename(columns={"Reporting year": "Measurement Year", "City": "City Name"})

temperature = pd.merge(temperature1, temperature2, on=["City Name", "Measurement Year", "Average annual temperature (in Celsius)​"], how="outer")

temperature = temperature.rename(columns={"Average annual temperature (in Celsius)​": "temperature", "Measurement Year":"year", "City Name":"id_city"})

temperature = temperature.dropna()

print(temperature.to_string())
#temperature.head()

# The year from this dataset: community_Wide_2017 - 2017_-_Cities_Community_Wide_Emissions.csv is not necessary the most accurate one, but the best one there is.
# The desired result is the year of the avg temperature, but the only available seems to be the reporting year, the accouting year, population year and GDP year.
# If another value for year is prefered, feel free to change


     temperature  year                                                   id_city
0            6.8  2009                                         Ville de Montreal
1            9.5  2013                                        Greater Manchester
2           10.4  2014                                              City of Reno
3            7.9  2014                                       City of Minneapolis
4            7.9  2014                                        City of London, ON
5           12.0  2013                                         Ville de Lausanne
6           11.6  2015                                          City of Columbus
8           12.7  2015                                 Incheon Metropolitan City
9            9.4  2010                                           City of Chicago
10          15.0  2015                                         City of Melbourne
12          15.0  2013                                      City of Richmond, VA
13          21.0  2014      

C:\Users\krist\AppData\Local\Temp\ipykernel_26928\412784116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temperature1["Measurement Year"] = pd.to_datetime(temperature1["Measurement Year"]).dt.year


In [4]:
temperature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 415
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperature  400 non-null    float64
 1   year         400 non-null    int64  
 2   id_city      400 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 12.5+ KB


In [8]:
population1 = citywide_ghg_2016[["Current Population", "Measurement Year", "City Name"]]

population2 = emissions_reduction_2017[["Population","Population Year", "City"]]

population1["Measurement Year"] = pd.to_datetime(population1["Measurement Year"]).dt.year

population2 = population2.rename(columns={"Population": "Current Population","Population Year":"Measurement Year", "City": "City Name"})

population = pd.merge(population1, population2, on=["City Name", "Measurement Year", "Current Population"], how="outer")

population = population.rename(columns={"Current Population": "population", "Measurement Year":"year", "City Name":"id_city"})

population = population.dropna()
#print(population.to_string())
#population.info()
population.head()

C:\Users\Bruger\AppData\Local\Temp\ipykernel_16232\1888771496.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population1["Measurement Year"] = pd.to_datetime(population1["Measurement Year"]).dt.year


,population,year,id_city
0,1886481.0,2009.0,Ville de Montreal
1,2762000.0,2013.0,Greater Manchester
2,236995.0,2014.0,City of Reno
3,407207.0,2014.0,City of Minneapolis
4,375000.0,2014.0,"City of London, ON"


In [9]:
#gdp1 = citywide_ghg_2016[["City GDP", "GDP Currency", "Year of GDP", "GDP Source"]]
#citywide_ghg_2016.head()
#gdp2 = community_Wide_2017[["GDP", "GDP Currency", "GDP Year", "GDP Source"]]
#gdp2.head()

In [16]:
gdp1 = citywide_ghg_2016[["City GDP", "GDP Currency", "Year of GDP", "GDP Source", "City Name"]]

gdp2 = community_Wide_2017[["GDP", "GDP Currency", "GDP Year", "GDP Source", "City"]]

gdp2 = gdp2.rename(columns={"GDP": "City GDP","GDP Year":"Year of GDP", "City":"City Name"})

gdp = pd.merge(gdp1, gdp2, on=["City Name", "Year of GDP", "City GDP", "GDP Currency", "GDP Source"], how="outer")

gdp = gdp.rename(columns={"City GDP": "gdp", "GDP Currency":"gdp_currency", "Year of GDP":"year", "GDP Source":"source", "City Name":"id_city"})

#gdp = gdp.dropna()
#print(gdp.to_string())
#gdp.info()
gdp.head()

,gdp,gdp_currency,year,source,id_city
0,1.201180e+11,CAD Canadian Dollar,2014.0,"Ville de Montréal, Montréal en statistiques, h...",Ville de Montreal
1,8.777300e+10,NaN,2013.0,UK ONS (using 1.56 USD/GBP) Figures provided i...,Greater Manchester
2,1.000000e+10,USD US Dollar,2014.0,Bureau of Economic Analysis10.4,City of Reno
3,2.357000e+11,USD US Dollar,2014.0,http://www.bea.gov/iTable/iTable.cfm?reqid=70&...,City of Minneapolis
4,1.540000e+10,CAD Canadian Dollar,2014.0,"Conference Board of Canada, London Census Metr...","City of London, ON"
